In [1]:
"""Bayesian neural network using variational inference
(see, e.g., Blundell et al. (2015); Kucukelbir et al. (2016)).
Inspired by autograd's Bayesian neural network example.
References
----------
http://edwardlib.org/tutorials/bayesian-neural-network
"""


import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal, PointMass, Empirical

In [2]:
def build_toy_dataset(N=40, noise_std=0.1):
  D = 1
  X = np.concatenate([np.linspace(0, 2, num=N / 2),
                      np.linspace(6, 8, num=N / 2)])
  y = np.cos(X) + np.random.normal(0, noise_std, size=N)
  X = (X - 4.0) / 4.0
  X = X.reshape((N, D))
  return X, y


def neural_network(X):
  h = tf.tanh(tf.matmul(X, W_0) + b_0)
  h = tf.tanh(tf.matmul(h, W_1) + b_1)
  h = tf.matmul(h, W_2) + b_2
  return tf.reshape(h, [-1])

In [3]:
ed.set_seed(42)

N = 40  # number of data points
D = 1   # number of features

# DATA
X_train, y_train = build_toy_dataset(N)

# MODEL
W_0 = Normal(loc=tf.zeros([D, 10]), scale=tf.ones([D, 10]))
W_1 = Normal(loc=tf.zeros([10, 10]), scale=tf.ones([10, 10]))
W_2 = Normal(loc=tf.zeros([10, 1]), scale=tf.ones([10, 1]))
b_0 = Normal(loc=tf.zeros(10), scale=tf.ones(10))
b_1 = Normal(loc=tf.zeros(10), scale=tf.ones(10))
b_2 = Normal(loc=tf.zeros(1), scale=tf.ones(1))

X = tf.placeholder(tf.float32, [N, D])
y = Normal(loc=neural_network(X), scale=0.1 * tf.ones(N))

# INFERENCE
qW_0 = Normal(loc=tf.Variable(tf.random_normal([D, 10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, 10]))))
qW_1 = Normal(loc=tf.Variable(tf.random_normal([10, 10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10, 10]))))
qW_2 = Normal(loc=tf.Variable(tf.random_normal([10, 1])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10, 1]))))
qb_0 = Normal(loc=tf.Variable(tf.random_normal([10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10]))))
qb_1 = Normal(loc=tf.Variable(tf.random_normal([10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10]))))
qb_2 = Normal(loc=tf.Variable(tf.random_normal([1])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))

In [5]:
prior_post_mapping = {W_0: qW_0, b_0: qb_0,
                     W_1: qW_1, b_1: qb_1,
                     W_2: qW_2, b_2: qb_2}

In [6]:
inference = ed.KLqp(prior_post_mapping, data={X: X_train, y: y_train})
inference.run()

1000/1000 [100%] ██████████████████████████████ Elapsed: 12s | Loss: 608.647


In [7]:
y_post = ed.copy(y, prior_post_mapping)

In [21]:
#y_post.variance
ed.evaluate('log_likelihood', data={y_post: x_train})

<bound method Normal.variance of <ed.RandomVariable 'copied/Normal_2/' shape=(40,) dtype=float32>>

In [9]:
print(ed.evaluate('mean_squared_error', data={X: X_train, y_post: y_train}))

InvalidArgumentError: Input 0 of node copied/copied/MatMul was passed double from Const_8:0 incompatible with expected float.

In [31]:
q={"a": 1, "b":2}
l={"i": 1, "l":2}
ll = dict(q, **l)

In [6]:
inference_HMC = ed.HMC([W_0, W_1, W_2, b_0, b_1, b_2], data={X: X_train, y: y_train})

inference_HMC.run()

10000/10000 [100%] ██████████████████████████████ Elapsed: 94s | Acceptance Rate: 0.000


In [9]:
inference_map = ed.MAP([W_0, W_1, W_2, b_0, b_1, b_2], data={X: X_train, y: y_train})

inference_map.run()

1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 111.985


In [12]:
x_test, y_test = build_toy_dataset(N)

In [16]:
x_post = ed.copy(X_train, {W_0: qW_0, b_0: qb_0,
                     W_1: qW_1, b_1: qb_1,
                     W_2: qW_2, b_2: qb_2})

# log-likelihood performance
ed.evaluate('log_likelihood', data={x_post: X_train})

#ed.ppc(inference_hmc, data={X:x_test, y:y_test}, latent_vars=None, n_samples=100)

TypeError: Could not copy instance: [[-1.        ]
 [-0.97368421]
 [-0.94736842]
 [-0.92105263]
 [-0.89473684]
 [-0.86842105]
 [-0.84210526]
 [-0.81578947]
 [-0.78947368]
 [-0.76315789]
 [-0.73684211]
 [-0.71052632]
 [-0.68421053]
 [-0.65789474]
 [-0.63157895]
 [-0.60526316]
 [-0.57894737]
 [-0.55263158]
 [-0.52631579]
 [-0.5       ]
 [ 0.5       ]
 [ 0.52631579]
 [ 0.55263158]
 [ 0.57894737]
 [ 0.60526316]
 [ 0.63157895]
 [ 0.65789474]
 [ 0.68421053]
 [ 0.71052632]
 [ 0.73684211]
 [ 0.76315789]
 [ 0.78947368]
 [ 0.81578947]
 [ 0.84210526]
 [ 0.86842105]
 [ 0.89473684]
 [ 0.92105263]
 [ 0.94736842]
 [ 0.97368421]
 [ 1.        ]]

In [13]:
N = 40  # number of data points
D = 1   # number of features

# DATA
X_train, y_train = build_toy_dataset(N)

# MODEL
W_0 = Normal(loc=tf.zeros([D, 10]), scale=tf.ones([D, 10]))
W_1 = Normal(loc=tf.zeros([10, 10]), scale=tf.ones([10, 10]))
W_2 = Normal(loc=tf.zeros([10, 1]), scale=tf.ones([10, 1]))
b_0 = Normal(loc=tf.zeros(10), scale=tf.ones(10))
b_1 = Normal(loc=tf.zeros(10), scale=tf.ones(10))
b_2 = Normal(loc=tf.zeros(1), scale=tf.ones(1))

X = tf.placeholder(tf.float32, [N, D])
y = Normal(loc=neural_network(X), scale=0.1 * tf.ones(N))

# INFERENCE
qW_0 = Empirical(params=tf.Variable(tf.random_normal([D,10])))
qW_1 = Empirical(params=tf.Variable(tf.random_normal([10,10])))
qW_2 = Empirical(params=tf.Variable(tf.random_normal([10,1])))
qb_0 = Empirical(params=tf.Variable(tf.random_normal([10])))
qb_1 = Empirical(params=tf.Variable(tf.random_normal([10])))
qb_2 = Empirical(params=tf.Variable(tf.random_normal([1])))

In [1]:
inference2 = ed.inference.HMC({W_0: qW_0, b_0: qb_0,
                     W_1: qW_1, b_1: qb_1,
                     W_2: qW_2, b_2: qb_2}, data={X: X_train, y: y_train})
inference2.run()

NameError: name 'ed' is not defined